In [1]:
def calculate_column_differences(data):
    # Create a copy to avoid modifying original data
    columns = list(map(list, zip(*data)))
    
    # Sort each column independently
    sorted_columns = [sorted(col) for col in columns]
    
    # Calculate differences between corresponding elements
    return sum(abs(x - y) for x, y in zip(*sorted_columns))

def calculate_weighted_frequencies(data):
    values, frequencies = map(list, zip(*data))
    return sum(value * frequencies.count(value) for value in values)

def main():
    # Read and parse input
    with open('./inputs/1.txt', 'r') as file:
        data = [list(map(int, line.split())) for line in file.read().splitlines()]
    
    # Part 1: Column differences
    total_difference = calculate_column_differences(data)
    print("Sum of absolute differences:", total_difference)
    
    # Part 2: Weighted frequencies
    weighted_sum = calculate_weighted_frequencies(data)
    print("Weighted sum:", weighted_sum)

if __name__ == "__main__":
    main()

Sum of absolute differences: 2176849
Weighted sum: 23384288


In [2]:
def read_sequences(filename: str) -> list[list[int]]:
    """Read and parse sequences from input file."""
    with open(filename, 'r') as file:
        return [list(map(int, line.split())) for line in file]

def is_safe_sequence(levels: list[int]) -> bool:
    """Check if a sequence is safe based on consecutive differences."""
    differences = [x - y for x, y in zip(levels, levels[1:])]
    return all(1 <= diff <= 3 for diff in differences) or all(-3 <= diff <= -1 for diff in differences)

def can_be_made_safe(levels: list[int]) -> bool:
    """Check if sequence can be made safe by removing one element."""
    # First check if it's already safe
    if is_safe_sequence(levels):
        return True
        
    # Try removing each element
    for i in range(len(levels)):
        modified = levels[:i] + levels[i + 1:]
        if is_safe_sequence(modified):
            return True
    return False

def main() -> None:
    filename = './inputs/2.txt'
    sequences = read_sequences(filename)
    
    # Part 1: Count safe sequences
    safe_count = sum(1 for seq in sequences if is_safe_sequence(seq))
    print(f"Safe sequence count (Part 1): {safe_count}")
    
    # Part 2: Count sequences that can be made safe
    modifiable_count = sum(1 for seq in sequences if can_be_made_safe(seq))
    print(f"Modifiable safe sequence count (Part 2): {modifiable_count}")

if __name__ == "__main__":
    main()

Safe sequence count (Part 1): 299
Modifiable safe sequence count (Part 2): 364


In [3]:
import re

def parse_mul_instructions(memory_input, part):
    # Regex patterns for different instructions
    mul_pattern = r'mul\((\d{1,3})\s*,\s*(\d{1,3})\)'
    do_pattern = r'do\(\)'
    dont_pattern = r'don\'*t\(\)'
    
    # Track multiplication state
    mul_enabled = True
    total_sum = 0
    
    # Find all instructions in order
    mul_matches = list(re.finditer(mul_pattern, memory_input))
    if part == 2:
        do_matches = list(re.finditer(do_pattern, memory_input))
        dont_matches = list(re.finditer(dont_pattern, memory_input))
        all_matches = sorted(
            mul_matches + do_matches + dont_matches, 
            key=lambda x: x.start()
        )
    else:
        all_matches = mul_matches
    
    # Process instructions in order
    for match in all_matches:
        if part == 2:
            if match.group() == 'do()':
                mul_enabled = True
            elif match.group() == "don't()":
                mul_enabled = False
        
        if match.group().startswith('mul(') and mul_enabled:
            x, y = match.groups()
            total_sum += int(x) * int(y)
    
    return total_sum

def main():
    # Read the input from a text file
    try:
        with open('./inputs/3.txt', 'r') as file:
            memory_input = file.read().strip()
        
        # Calculate the sum of multiplication results for Part 1
        result_part1 = parse_mul_instructions(memory_input, part=1)
        print(f"Part 1 - Sum of multiplication results: {result_part1}")
        
        # Calculate the sum of multiplication results for Part 2
        result_part2 = parse_mul_instructions(memory_input, part=2)
        print(f"Part 2 - Sum of multiplication results: {result_part2}")
    
    except FileNotFoundError:
        print("Error: 'corrupted_memory.txt' file not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Part 1 - Sum of multiplication results: 159833790
Part 2 - Sum of multiplication results: 89349241


In [4]:
from typing import List, Tuple, Set
from dataclasses import dataclass

@dataclass(frozen=True)
class GridCoord:
    row: int
    col: int

    def offset_by(self, direction: Tuple[int, int], multiplier: int = 1) -> 'GridCoord':
        dr, dc = direction
        return GridCoord(
            self.row + (dr * multiplier),
            self.col + (dc * multiplier)
        )

# Constants
DIRECTIONS = [(dr, dc) for dr in [-1, 0, 1] for dc in [-1, 0, 1] if not (dr == dc == 0)]
VALID_MMSS_PATTERNS = {"MMSS", "MSSM", "SSMM", "SMMS"}

def read_grid(filename: str) -> List[str]:
    """
    Reads and processes a grid from a file.
    
    Args:
        filename: Path to the input file
        
    Returns:
        List of strings representing rows in the grid
    """
    try:
        with open(filename, 'r') as file:
            return [line.strip().upper() for line in file.readlines()]
    except FileNotFoundError:
        print(f"The file {filename} was not found.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

def is_valid_position(pos: GridCoord, grid: List[str]) -> bool:
    """Check if a position is within grid bounds."""
    return 0 <= pos.row < len(grid) and 0 <= pos.col < len(grid[0])

def count_xmas(grid: List[str]) -> int:
    """
    Counts occurrences of 'XMAS' in all directions.
    
    Searches for 'XMAS' patterns starting from each 'X' in the grid,
    checking all 8 possible directions.
    """
    count = 0
    
    for r in range(len(grid)):
        for c in range(len(grid[0])):
            if grid[r][c] != "X":
                continue
                
            current_pos = GridCoord(r, c)
            for direction in DIRECTIONS:
                # Check if all positions for 'MAS' are valid
                positions = [current_pos.offset_by(direction, i) for i in range(1, 4)]
                if not all(is_valid_position(pos, grid) for pos in positions):
                    continue
                
                # Check if letters match 'MAS'
                if (grid[positions[0].row][positions[0].col] == "M" and
                    grid[positions[1].row][positions[1].col] == "A" and
                    grid[positions[2].row][positions[2].col] == "S"):
                    count += 1
                    
    return count

def count_mmss(grid: List[str]) -> int:
    """
    Counts occurrences of valid MMSS patterns around each 'A'.
    
    Checks the four corners around each 'A' for valid MMSS combinations.
    """
    count = 0
    
    for r in range(1, len(grid) - 1):
        for c in range(1, len(grid[0]) - 1):
            if grid[r][c] != "A":
                continue
                
            corners = [
                grid[r - 1][c - 1],  # top-left
                grid[r - 1][c + 1],  # top-right
                grid[r + 1][c + 1],  # bottom-right
                grid[r + 1][c - 1]   # bottom-left
            ]
            if "".join(corners) in VALID_MMSS_PATTERNS:
                count += 1
                
    return count

def main() -> None:
    filename = 'inputs/4.txt'
    grid = read_grid(filename)
    
    if grid:
        xmas_count = count_xmas(grid)
        mmss_count = count_mmss(grid)
        
        print(f"XMAS Count: {xmas_count}")
        print(f"MMSS Count: {mmss_count}")
    else:
        print("No grid data available.")

if __name__ == "__main__":
    main()

XMAS Count: 2414
MMSS Count: 1871


In [7]:
def parse_input(filename: str) -> tuple[dict[int, set[int]], list[list[int]]]:
    """
    Parse input file into rules and updates.
    
    Args:
        filename: Path to input file
        
    Returns:
        Tuple of (rules_dict, updates_list) where:
        - rules_dict: Maps page number to set of pages that must come after it
        - updates_list: List of updates, each containing list of page numbers
    """
    with open(filename, 'r') as file:
        rules_section, updates_section = file.read().strip().split('\n\n')
    
    # Parse rules into a directed graph
    rules = {}
    for line in rules_section.splitlines():
        before, after = map(int, line.split('|'))
        rules.setdefault(before, set()).add(after)
    
    # Parse updates into lists of integers
    updates = [list(map(int, line.split(','))) for line in updates_section.splitlines()]
    
    return rules, updates

def is_valid_order(pages: list[int], rules: dict[int, set[int]]) -> bool:
    """Check if pages are in valid order according to rules."""
    for i, page in enumerate(pages):
        if page not in rules:
            continue
        for after_page in rules[page]:
            for j in range(i):
                if pages[j] == after_page:
                    return False
    return True

def find_correct_order(pages: list[int], rules: dict[int, set[int]]) -> list[int]:
    """Find the correct order for a list of pages using topological sort."""
    # Build adjacency list for these specific pages
    graph = {page: set() for page in pages}
    for page in pages:
        if page in rules:
            graph[page].update(after for after in rules[page] if after in pages)
    
    # Calculate in-degrees
    in_degree = {page: 0 for page in pages}
    for page in pages:
        for after in graph[page]:
            in_degree[after] += 1
    
    # Topological sort
    result = []
    queue = [page for page in pages if in_degree[page] == 0]
    
    while queue:
        page = queue.pop(0)
        result.append(page)
        for after in graph[page]:
            in_degree[after] -= 1
            if in_degree[after] == 0:
                queue.append(after)
    
    return result

def solve_page_ordering(filename: str) -> tuple[int, int]:
    """
    Solve both parts of the page ordering problem.
    
    Returns:
        Tuple of (part1_result, part2_result)
    """
    rules, updates = parse_input(filename)
    
    # Part 1: Sum middle pages of valid updates
    valid_sum = 0
    invalid_sum = 0
    
    print("\nProcessing updates:")
    print("-" * 50)
    
    for i, update in enumerate(updates, 1):
        if is_valid_order(update, rules):
            middle_index = len(update) // 2
            valid_sum += update[middle_index]
            print(f"Update {i}: VALID")
            print(f"Pages: {update}")
            print(f"Middle page: {update[middle_index]}")
        else:
            corrected = find_correct_order(update, rules)
            middle_index = len(corrected) // 2
            invalid_sum += corrected[middle_index]
            print(f"Update {i}: INVALID")
            print(f"Original: {update}")
            print(f"Corrected: {corrected}")
            print(f"Middle page: {corrected[middle_index]}")
        print()
    
    return valid_sum, invalid_sum

if __name__ == "__main__":
    part1, part2 = solve_page_ordering('inputs/5.txt')
    print("=" * 50)
    print(f"Part 1 - Sum of middle pages (valid updates): {part1}")
    print(f"Part 2 - Sum of middle pages (corrected invalid updates): {part2}")


Processing updates:
--------------------------------------------------
Update 1: INVALID
Original: [49, 58, 38, 28, 69, 66, 48, 78, 46, 56, 87, 63, 99, 75, 27, 96, 94, 11, 71, 24, 32]
Corrected: [96, 24, 28, 99, 71, 63, 49, 46, 56, 66, 32, 69, 94, 38, 78, 48, 58, 75, 11, 87, 27]
Middle page: 32

Update 2: INVALID
Original: [52, 32, 99, 49, 94, 78, 46, 66, 75, 17, 58, 57, 69, 87, 28, 96, 24, 56, 27, 11, 48, 38, 71]
Corrected: [96, 24, 57, 28, 52, 99, 71, 49, 46, 17, 56, 66, 32, 69, 94, 38, 78, 48, 58, 75, 11, 87, 27]
Middle page: 66

Update 3: VALID
Pages: [52, 99, 71, 49, 46, 17, 56, 66, 38, 78, 75, 27, 89, 44, 42]
Middle page: 66

Update 4: INVALID
Original: [58, 71, 56, 38, 87, 94, 11, 13, 75, 99, 27, 78, 44, 63, 89, 42, 32, 46, 66, 69, 49]
Corrected: [99, 71, 63, 49, 46, 56, 66, 32, 69, 94, 38, 78, 58, 75, 11, 87, 27, 89, 44, 42, 13]
Middle page: 38

Update 5: VALID
Pages: [94, 48, 89, 42, 13, 37, 36, 12, 55]
Middle page: 13

Update 6: VALID
Pages: [49, 46, 17, 56, 66, 32, 69, 94, 